In [1]:
from glob import glob
import h5py
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [7]:
!pwd

/global/cfs/cdirs/dune/users/roberto


In [2]:
directory_proc = "/project/projectdirs/dune/data/larpix/processed_data/prod_20_10_12/"
filepaths_proc = glob(directory_proc+"*.h5")
filenames_proc = list(map(os.path.basename, filepaths_proc))

In [3]:
filename = filenames_proc[0]
f = h5py.File(directory_proc+filename,'r')

In [4]:
%matplotlib widget
hit_refs = f['events'][f['events']['nhit'] > 15]['hit_ref']
events = f['events'][f['events']['nhit'] > 15]['evid']
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection='3d')

tot_x = np.array([])
tot_q = np.array([])
tot_y = np.array([])
tot_rel_ts = np.array([])

for event_idx, hit_ref in zip(events[6:7], hit_refs[6:7]):
    hit_x = f['hits'][hit_ref]['px'] 
    hit_y = f['hits'][hit_ref]['py'] 
    hit_q = f['hits'][hit_ref]['q'] 
    hit_rel_ts = f['hits'][hit_ref]['ts'] - f['events'][event_idx]['ts_start']
    mask = (hit_q>0)&(hit_rel_ts <= 200)
    tot_x = np.concatenate((tot_x, hit_x[mask]), axis=None)
    tot_q = np.concatenate((tot_q, hit_q[mask]), axis=None)
    tot_y = np.concatenate((tot_y, hit_y[mask]), axis=None)
    tot_rel_ts = np.concatenate((tot_rel_ts, hit_rel_ts[mask]), axis=None)

s = ax.scatter(tot_x,tot_y,tot_rel_ts*100e-9*1e6,c=tot_q,alpha=1,cmap='Spectral_r')
ax.set_xlabel("x [mm]")
ax.set_ylabel("y [mm]")
ax.set_zlabel("t [$\mathrm{\mu}$s]")
fig.colorbar(s,ax=ax,label='Charge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
xyz = np.dstack((tot_x,tot_y,tot_rel_ts*100e-9*1e6))
xyz = xyz.reshape(xyz.shape[1],3)

In [6]:
from skimage.measure import LineModelND, ransac
from sklearn.decomposition import PCA

# robustly fit line only using inlier data with RANSAC algorithm
model_robust, inliers = ransac(xyz, LineModelND, min_samples=2,
                               residual_threshold=4, max_trials=1000)
outliers = inliers == False

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xyz[inliers][:, 0], xyz[inliers][:, 1], xyz[inliers][:, 2], c='b',
           marker='o', label='Inlier data')
ax.scatter(xyz[outliers][:, 0], xyz[outliers][:, 1], xyz[outliers][:, 2], c='r',
           marker='o', label='Outlier data')

X = xyz[inliers]

pca = PCA(n_components=1)
pca.fit(X)

## New code below
p = pca.components_
centroid = np.mean(X, 0)
segments = np.linspace(-100,100,10)[:, np.newaxis] * p
Xlen = X.shape[0]
avgPointCloud = 1 / Xlen * np.array([np.sum(X[:, 0]), np.sum(X[:, 1]), np.sum(X[:, 2])])
Xmean = X - avgPointCloud

cov = 1 / Xlen * X.T.dot(Xmean)
t = np.linspace(-0.15,0.15,10)
linearReg = avgPointCloud + cov[:, 0] * np.vstack(t)
lineplot = ax.plot(*(centroid + segments).T, color="red", label='Linear regression')

ax.legend(loc='lower left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …